In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

import datetime as dt

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from thefuzz import process

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

from sklearn.metrics import make_scorer


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

# Lets use fuzzy matching to create a mapping between the country names in the hospital beds dataset and the location dataset
from thefuzz import process

import csv

from scipy.stats import entropy

#from datetime import datetime as dt

import joblib

# Disable warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def create_submission_file(y_preds, file_name): 
    with open(file_name, "w") as csvfile: 
        wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL) 
        wr.writerow(["Id", "Prediction"]) 
        for i, pred in enumerate(y_preds): 
            wr.writerow([str(i), str(pred)])

# Data Visualization

In [ ]:
cases_train_df = pd.read_csv('data/cases_2021_train.csv')#.dropna(subset=['date_confirmation'])
cases_test_df = pd.read_csv('data/cases_2021_test.csv')#.dropna(subset=['date_confirmation'])
cases_df = pd.concat([cases_train_df, cases_test_df])
locations_df = pd.read_csv('data/location_2021.csv')
country_continent_df = pd.read_csv('data/continents-according-to-our-world-in-data.csv')
country_continent_df.rename(columns={'Entity':'country', 'Continent':'continent', 'Code':'code'}, inplace=True)
country_continent_df = country_continent_df[['country', 'code', 'continent']]
pop_df = pd.read_csv('population.csv')
pop_df.rename(columns={'Country Name':'country', '2022':'population'}, inplace=True)

In [ ]:
# Group locations_df by country and sum the values
locations_df = locations_df.groupby('Country_Region').sum().reset_index()

In [ ]:
# Importing data set containing country, continent pairs and merging with the location data
# https://ourworldindata.org/grapher/continents-according-to-our-world-in-data
country_continent_df.rename(columns={'Entity':'country', 'Continent':'continent', 'Code':'code'}, inplace=True)
country_continent_df = country_continent_df[['country', 'code', 'continent']]
country_continent_df.head(5)

In [ ]:
# Checking for countries that are not in the external data set
unique_country_names = pd.DataFrame(pd.unique(locations_df['Country_Region']))
unique_country_names.columns = ['country']
matches = unique_country_names.merge(country_continent_df, left_on='country', right_on='country', how='left')
# check for missing values in continent column
print(matches[matches['continent'].isna()])

There is a mismatch in the naming scheme between the two datasets. We can use the following mappings to get a better merge.
- US -> United States x
- Micronesia (Country) -> Micronesia x
- Holy See -> Vatican x
- Burma -> Myanmar x
- Taiwan* -> Taiwan x
- Congo (Brazzaville), Congo (Kinshasa)	-> Congo x
- Diamond See -> Cruise x
- MS Zaandam -> Cruise
- Cabo Verde -> Cabo Verde (This was manually added to the dataset .csv file)
- Korea, South -> South Korea x
- Timor-Leste -> Timor-Leste(This was manually added to the dataset .csv file)
- West Bank and Gaza -> Palestine x

We can manually match these.

In [ ]:
def clean_location_data(df):
    df.rename(columns={'Country_Region':'country', 'Province_State':'state', 'Lat':'lat', 'Long':'long'}, inplace=True)
    # Cleaning country names
    df['country'] = df['country'].str.replace('US', 'United States', regex=False)
    df['country'] = df['country'].str.replace('Korea, South', 'South Korea', regex=False)
    df['country'] = df['country'].str.replace('Taiwan*', 'Taiwan', regex=False)
    df['country'] = df['country'].str.replace('Micronesia (Country)', 'Micronesia', regex=False)
    df['country'] = df['country'].str.replace('Holy See', 'Vatican', regex=False)
    df['country'] = df['country'].str.replace('West Bank and Gaza', 'Palestine', regex=False)
    df['country'] = df['country'].str.replace('Diamond See', 'International Waters', regex=False)
    df['country'] = df['country'].str.replace('Congo (Brazzaville)', 'Republic of Congo', regex=False)
    df['country'] = df['country'].str.replace('Congo (Kinshasa)', 'Democratic Republic of Congo', regex=False)
    df['country'] = df['country'].str.replace('Burma', 'Myanmar', regex=False)
    
    return df

In [ ]:
unique_country_names = pd.DataFrame(pd.unique(clean_location_data(locations_df)['country']))
unique_country_names.columns = ['country']
matches = unique_country_names.merge(country_continent_df, left_on='country', right_on='country', how='left')
# check for missing values in continent column
print(matches[matches['continent'].isna()])

In [ ]:
locations_df = clean_location_data(locations_df)
# Merging the dataframes
locations_df = locations_df.merge(country_continent_df, left_on='country', right_on='country', how='left')

In [ ]:
cases_df_grouped = cases_df.groupby('country').size()

# Now lets merge the dataframes to get the population for each country
cases_df_grouped = cases_df_grouped.reset_index()
cases_df_grouped.rename(columns={0:'cases'}, inplace=True)

In [ ]:
cases_df_grouped = cases_df_grouped.merge(locations_df, on='country', how='left')
cases_df_grouped = cases_df_grouped.merge(pop_df, left_on='country', right_on='country', how='left')[['country','continent','cases','Confirmed','Case_Fatality_Ratio','population']]
cases_df_grouped2 = cases_df_grouped.copy()

In [ ]:
# Grouping by continent and summing the values
cases_df_grouped = cases_df_grouped.groupby('continent').sum().reset_index().sort_values('Confirmed', ascending=False)
cases_df_grouped

In [ ]:
# Get a diverging blue and red color map
cmap = plt.get_cmap('bwr')
colors = cmap(np.linspace(1, 0, len(cases_df_grouped)))

In [ ]:
# Plotting a bar chart of the number of cases per continent
plt.figure(figsize=(10, 4))
plt.suptitle('Data Availability per Continent')
plt.subplot(121)
plt.bar(cases_df_grouped['continent'], cases_df_grouped['Confirmed'] / 10**6, color=colors)
plt.title('Number of confirmed cases in locations dataset')
plt.ylabel('Confirmed cases, millions')
plt.xlabel('Continent')
plt.xticks(rotation=45)
# Change ytics to scientific notation
plt.ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.subplot(122)
plt.bar(cases_df_grouped['continent'], cases_df_grouped['cases'] / 10**6, color=colors)
plt.title('Number of cases in cases dataset')
plt.xlabel('Continent')
plt.xticks(rotation=45)
plt.ticklabel_format(style='scientific', axis='y')
plt.tight_layout()
plt.savefig('plots/continent_cases.svg')
plt.show()

In [ ]:
cases_df_grouped2['data_availability_confirmed'] = cases_df_grouped2['cases'] / cases_df_grouped2['Confirmed']* 100
cases_df_grouped2['data_availability_population'] = cases_df_grouped2['cases'] / cases_df_grouped2['population'] * 100
# Replace Nan values with 0
cases_df_grouped2.fillna(0, inplace=True)

In [ ]:
gdf = gpd.GeoDataFrame(cases_df_grouped2)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merging the geodataframe with the population dataframe
world = world.merge(gdf, left_on='name', right_on='country', how='left')

# Plotting the map
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
world.boundary.plot(ax=ax)
world.plot(column='data_availability_population', ax=ax, legend=True, cmap='viridis')
plt.title('Data Availability per Country', size=24)
# Remove axis
ax.set_yticklabels([])
ax.set_xticklabels([])
# Remove outline
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
# Increase colorbar font size
cbar = ax.get_figure().get_axes()[1]
cbar.tick_params(labelsize=15)
plt.savefig('plots/data_availability_map.svg')
plt.show()

# Data Preprocessing

In [ ]:
cases_df = pd.read_csv('data/cases_2021_train.csv')
locations_df = pd.read_csv('data/location_2021.csv')

In [ ]:
def processDateTime(dateTimeSeries):
    
    if pd.isna(dateTimeSeries):
        return dt.date(2000,1,1)
    elif (dateTimeSeries == '25.02.2020 - 26.02.2020'):
        return dt.date(2020,2,25)
    
    dateArray = dateTimeSeries.split('.')
    
    day = int(dateArray[0])
    month = int(dateArray[1])
    year = int(dateArray[2])

    dateVal = dt.date(year, month, day)
    return dateVal

In [ ]:
def categorizeOutcome(outcome):
    
    outcomeDictionary = {'hospitalized' : 'Hospitalized/Under Treatment', 
                         'recovered' : 'Recovered/Discharged', 
                         'deceased' : 'Deceased', 
                         'discharged' : 'Recovered/Discharged', 
                         'alive' : 'Recovered/Discharged',
                         'discharge' : 'Recovered/Discharged', 
                         'under treatment' : 'Hospitalized/Under Treatment', 
                         'stable' : 'Recovered/Discharged', 
                         'died' : 'Deceased',
                         'receiving treatment' : 'Hospitalized/Under Treatment', 
                         'death' : 'Deceased', 
                         'stable condition' : 'Recovered/Discharged', 
                         'dead' : 'Deceased',
                         'discharged from hospital' : 'Recovered/Discharged', 
                         'critical condition' : 'Hospitalized/Under Treatment',
                         'released from quarantine' : 'Recovered/Discharged', 
                         'recovering at home 03.03.2020' : 'Recovered/Discharged'}
    
    return outcomeDictionary.get(outcome)

In [ ]:
def convertPosition2Rad(latitude,longitude):
    conversion = np.deg2rad([latitude,longitude])
    return conversion[0], conversion[1];

In [ ]:
def processAge(rawAge):
    if (pd.isna(rawAge)):
        return np.nan
    
    if (len(rawAge) <= 2):
        return int(rawAge)
    elif '-' not in rawAge:
        return round(float(rawAge))
    else:
        return np.nan

In [ ]:
def pipelinePreprocess(case_df, location_df):
    # Merge primary dataset w/ provincial dataset
    preprocessed_df = pd.merge(left=case_df, right=location_df, left_on=['country', 'province'], right_on=['Country_Region', 'Province_State'], how='left')
    # Clean up column names
    preprocessed_df = preprocessed_df.rename(columns={'Lat':'province_latitude', 'Long_':'province_longitude', 'Confirmed':'confirmed', 'Deaths':'deaths', 
                            'Recovered':'recovered', 'Active':'active', 'Combined_Key':'combined_key', 'Incident_Rate':'incident_rate', 
                            'Case_Fatality_Ratio':'Expected_Mortality_Rate'})
    # Select relevant columns
    preprocessed_df = preprocessed_df[['age', 'sex', 'province', 'country', 'latitude', 'longitude', 'date_confirmation', 'source','chronic_disease_binary', 'outcome', 
             'province_latitude', 'province_longitude', 'confirmed', 'deaths', 'recovered', 'active','combined_key', 'incident_rate', 'Expected_Mortality_Rate']]
    # Categorize outcomes into the most salient
    preprocessed_df['outcome'] = preprocessed_df['outcome'].str.lower()
    preprocessed_df['outcome'] = preprocessed_df['outcome'].apply(lambda x: categorizeOutcome(x))
    # Cleanup date_confirmation
    preprocessed_df['date_confirmation'] = preprocessed_df.apply(lambda x: processDateTime(x['date_confirmation']), axis=1)
    # Convert coordinates from degrees to radians
    preprocessed_df['latitude_r'] = preprocessed_df['latitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['longitude_r'] = preprocessed_df['longitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['province_latitude_r'] = preprocessed_df['province_latitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['province_longitude_r'] = preprocessed_df['province_longitude'].apply(lambda x: np.deg2rad(x))

    # Process age
    preprocessed_df['age'] = preprocessed_df['age'].apply(lambda x: processAge(x))

    # Impute missing values using Iterative Imputer
    imputer = IterativeImputer(max_iter=10, random_state=0)

    # Impute missing values for age
    preprocessed_df['age'] = imputer.fit_transform(preprocessed_df[['age']])
    preprocessed_df['age'] = preprocessed_df['age'].apply(lambda x: round(x))
    # Map sex to binary
    # Mapping sex to a number
    sexMap = {'male':0, 'female':1}
    preprocessed_df['sex'] = preprocessed_df['sex'].map(sexMap)
    # Impute sex
    preprocessed_df['sex'] = imputer.fit_transform(preprocessed_df[['sex']])
    preprocessed_df['sex'] = preprocessed_df['sex'].apply(lambda x: round(x))
    
    # One hot encode country and province, then use iterative imputer to fill in missing values
    preprocessed_df = pd.get_dummies(preprocessed_df, columns=['country', 'province'])
    
    return preprocessed_df

In [ ]:
training_df = pipelinePreprocess(cases_df, locations_df)

# Tasks 3 - 4

## Exporting external data

### Hopital beds (per 10k) -> hospital_beds_per_10k

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/hospital-beds-(per-10-000-population)

### Population Density and Income -> pop_density_per_sqkm, country_income

https://data.worldbank.org/indicator/EN.POP.DNST?type=shaded&view=map&year=2021

In [ ]:
train_o = pd.read_excel('data/cases_2021_train_processed_2.xlsx').dropna(subset=['outcome_group'])
test_o = pd.read_excel('data/cases_2021_test_processed_unlabelled_2.xlsx')
locations = pd.read_csv('data/location_2021.csv')
hospitalBeds = pd.read_excel('data/hospital_beds_per_10k.xlsx', skiprows=2)
countryIncome = pd.read_csv('data/country_income.csv')
popDensity = pd.read_csv('data/pop_density_per_sqkm.csv', skiprows=4)

In [ ]:
def pipelinePreprocess2(case_df, location_df, hospital_beds_df, country_income_df, pop_density_df, train=True, encoder=None, imputer=None, scaler=None):
    
    def haversine(lat1, lon1, lat2, lon2, radians=False):
        R = 6371
        if not radians:
            lat1 = np.deg2rad(lat1)
            lon1 = np.deg2rad(lon1)
            lat2 = np.deg2rad(lat2)
            lon2 = np.deg2rad(lon2)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.deg2rad(lat1)) * np.cos(np.deg2rad(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        return d
    
    # Drop unnecessary columns
    location_df = location_df.drop(columns=['Last_Update'])
    # Drop redundant columns
    location_df = location_df.drop(columns=['Recovered', 'Active', 'Incident_Rate', 'Confirmed', 'Deaths', 'Case_Fatality_Ratio'])
    # Merge primary dataset w/ provincial dataset
    preprocessed_df = pd.merge(left=case_df, right=location_df, left_on=['country', 'province'], right_on=['Country_Region', 'Province_State'], how='left')
    # Drop redundant columns
    preprocessed_df = preprocessed_df.drop(columns=['Country_Region', 'Province_State'])
    # Clean up column names
    preprocessed_df = preprocessed_df.rename(columns={'Lat':'province_latitude', 'Long_':'province_longitude', 'Confirmed':'confirmed', 'Deaths':'deaths', 
                            'Recovered':'recovered', 'Active':'active', 'Combined_Key':'combined_key', 'Incident_Rate':'incident_rate', 
                            'Case_Fatality_Ratio':'case_fatality_ratio'})
    
    # Creating a mapping between the country names in the hospital beds dataset and the location dataset with the matching score
    # If the score is above some threshold, we will consider it a match
    # If the score is better than the previous match, we will replace the previous match
    
    hospital_country_mapping = {}
    for country in pd.unique(case_df['country']):
        match, score, _ = process.extractOne(country, hospital_beds_df['Location'])
        if score > 86:
            if country not in hospital_country_mapping:
                hospital_country_mapping[country] = (match, score)
            else:
                if score > hospital_country_mapping[country][1]:
                    hospital_country_mapping[country] = (match, score)
        else:
            print(f'No match for {country}')

    # Add the rest of the countries that did not match
    hospital_country_mapping['South Korea'] = ('Republic of Korea', 100)
    hospital_country_mapping['San Marino'] = ('Italy', 100)
        
    # Lets use the mapping to create a new column in the hospital beds dataset that matches the country names in the case_df dataset
    hospital_beds_df = hospital_beds_df[['Location', 'Tooltip']].rename(columns={'Location':'country', 'Tooltip':'hospital_beds_per_10k'})
    hospital_beds_df['country'] = hospital_beds_df['country'].apply(lambda x: hospital_country_mapping[x][0] if x in hospital_country_mapping else x)

    # Merge the hospital beds dataset with the case_df dataset
    preprocessed_df = pd.merge(left=preprocessed_df, right=hospital_beds_df, left_on='country', right_on='country', how='left', validate='m:1')

    # Doing the same for population density
    pop_density_df = pop_density_df[['Country Name', '2019']].rename(columns={'Country Name':'country', '2019':'pop_density_per_sqkm'})

    pop_density_mapping = {}
    for country in pd.unique(case_df['country']):
        match, score, _ = process.extractOne(country, pop_density_df['country'])
        if score > 89:
            if country not in pop_density_mapping:
                pop_density_mapping[country] = (match, score)
            else:
                if score > pop_density_mapping[country][1]:
                    pop_density_mapping[country] = (match, score)
        else:
            print(f'No match for {country}')

    pop_density_mapping['South Korea'] = ('Korea, Rep.', 100)

    pop_density_df['country'] = pop_density_df['country'].apply(lambda x: pop_density_mapping[x][0] if x in pop_density_mapping else x)
    preprocessed_df = pd.merge(left=preprocessed_df, right=pop_density_df, left_on='country', right_on='country', how='left', validate='m:1')

    # Doing the same for country income
    country_income_df = country_income_df.rename(columns={'Country':'country', 'IncomeGroup':'income_group'}).drop(columns=['Country Code'])

    income_mapping = {}
    for country in pd.unique(case_df['country']):
        match, score, _ = process.extractOne(country, pop_density_df['country'])
        if score > 89:
            if country not in income_mapping:
                income_mapping[country] = (match, score)
            else:
                if score > income_mapping[country][1]:
                    income_mapping[country] = (match, score)
        else:
            print(f'No match for {country}')

    income_mapping['South Korea'] = ('Korea, Rep.', 100)

    country_income_df['country'] = country_income_df['country'].apply(lambda x: income_mapping[x][0] if x in income_mapping else x)
    # Mapping the income group to a number
    incomeMap = {'Low income':0, 'Lower middle income':1, 'Middle income':2, 'Upper middle income':3, 'High income':4}
    country_income_df['income_group'] = country_income_df['income_group'].map(incomeMap)
    country_income_df['income_group'] = country_income_df['income_group'].fillna(-1)
    preprocessed_df = pd.merge(left=preprocessed_df, right=country_income_df, left_on='country', right_on='country', how='left', validate='m:1')

    # If the lat and long are greater than 90 and 180 respectively, we will assume that they are outliers and replace them with the mean of the country
    preprocessed_df['latitude'] = preprocessed_df['latitude'].apply(lambda x: x if x < 90 else np.nan)
    preprocessed_df['longitude'] = preprocessed_df['longitude'].apply(lambda x: x if x < 180 else np.nan)
    preprocessed_df['latitude'] = preprocessed_df['latitude'].fillna(preprocessed_df.groupby('country')['latitude'].transform('mean'))
    preprocessed_df['longitude'] = preprocessed_df['longitude'].fillna(preprocessed_df.groupby('country')['longitude'].transform('mean'))
    

    preprocessed_df['latitude'] = preprocessed_df['latitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['longitude'] = preprocessed_df['longitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['province_latitude'] = preprocessed_df['province_latitude'].apply(lambda x: np.deg2rad(x))
    preprocessed_df['province_longitude'] = preprocessed_df['province_longitude'].apply(lambda x: np.deg2rad(x))


    # Calculate the distance between the case location and the province capital
    preprocessed_df['distance_to_province_capital'] = haversine(preprocessed_df['latitude'], preprocessed_df['longitude'], preprocessed_df['province_latitude'], preprocessed_df['province_longitude'], radians=True)

    # If the province column is nan, replace the distance_to_province_capital with 0
    # Then add a new column called no_province which is 1 if the province is nan and 0 otherwise
    def no_province(row):
        if pd.isna(row['province']):
            return 1
        else:
            return 0
        
    preprocessed_df['no_province'] = preprocessed_df.apply(no_province, axis=1)
        
    # Mapping sex to a number
    sexMap = {'male':0, 'female':1}
    preprocessed_df['sex'] = preprocessed_df['sex'].map(sexMap)

    # One hot encoding province and country
    #preprocessed_df = pd.get_dummies(preprocessed_df, columns=['province', 'country'])

    # Drop unnecessary columns
    # Combined key
    preprocessed_df = preprocessed_df.drop(columns='combined_key')
    # Date confirmation
    # We will add the month of the date confirmation as a feature

    preprocessed_df['date_confirmation'] = pd.to_datetime(preprocessed_df['date_confirmation'])

    preprocessed_df['month_confirmation'] = preprocessed_df['date_confirmation'].dt.month

    preprocessed_df = preprocessed_df.drop(columns='date_confirmation')

    # Mapping outcome group to a number
    if train:
        outcomeMap = {'deceased':0, 'hospitalized':1, 'nonhospitalized':2}
        preprocessed_df['outcome_group'] = preprocessed_df['outcome_group'].map(outcomeMap)

    VAR_NUM = ['age','latitude','longitude','confirmed','deaths','recovered','active','incident_rate',
           'case_fatality_ratio', 'province_latitude', 'province_longitude', 'hospital_beds_per_10k', 
           'pop_density_per_sqkm', 'distance_to_province_capital']

    VAR_1HE = ['province', 'country']

    VAR_ELSE = ['sex', 'chronic_disease_binary', 'no_province', 'income_group', 'month_confirmation']

    if train:

        outcomes = preprocessed_df['outcome_group']

        # Rearrange columns
        preprocessed_df = preprocessed_df[VAR_NUM + VAR_1HE + VAR_ELSE]

        # One hot encoding province and country using SKLearn's 1 hot encoder
        # We will use the infrequent_if_exist handle_unknown parameter to encode unseen categories as all zeros
        encoder = OneHotEncoder(handle_unknown='infrequent_if_exist')
        encoder.fit(preprocessed_df[['province', 'country']])
        encoded = encoder.transform(preprocessed_df[['province', 'country']]).toarray()
        encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

        preprocessed_df = pd.concat([preprocessed_df, encoded_df], axis=1)
        preprocessed_df = preprocessed_df.drop(columns=['province', 'country'])

        # Iterative imputer to fill in missing values
        imputer = IterativeImputer(random_state=42, n_nearest_features=5, max_iter=15)
        imputer.fit(preprocessed_df)
        imputed = imputer.transform(preprocessed_df)
        preprocessed_df = pd.DataFrame(imputed, columns=preprocessed_df.columns)

        # MinMaxScaler to scale the numerical columns
        scaler = MinMaxScaler()
        scaler.fit(preprocessed_df[VAR_NUM])
        scaled = scaler.transform(preprocessed_df[VAR_NUM])
        scaled_df = pd.DataFrame(scaled, columns=VAR_NUM)

        preprocessed_df = preprocessed_df.drop(columns=VAR_NUM)
        preprocessed_df = pd.concat([preprocessed_df, scaled_df], axis=1)

        preprocessed_df['outcome_group'] = outcomes

        return preprocessed_df, encoder, imputer, scaler
    
    else:
        #VAR_ELSE.remove('outcome_group')
        # Rearrange columns
        preprocessed_df = preprocessed_df[VAR_NUM + VAR_1HE + VAR_ELSE]

        # One hot encoding province and country using the encoder
        encoded = encoder.transform(preprocessed_df[['province', 'country']]).toarray()
        encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

        preprocessed_df = pd.concat([preprocessed_df, encoded_df], axis=1)
        preprocessed_df = preprocessed_df.drop(columns=['province', 'country'])

        # Impute missing values
        imputed = imputer.transform(preprocessed_df)
        preprocessed_df = pd.DataFrame(imputed, columns=preprocessed_df.columns)

        # Scale numerical columns
        scaled = scaler.transform(preprocessed_df[VAR_NUM])
        scaled_df = pd.DataFrame(scaled, columns=VAR_NUM)

        preprocessed_df = preprocessed_df.drop(columns=VAR_NUM)
        preprocessed_df = pd.concat([preprocessed_df, scaled_df], axis=1)
        
        return preprocessed_df


train, e, i, s = pipelinePreprocess2(train_o, locations, hospitalBeds, countryIncome, popDensity, True)
test  = pipelinePreprocess2(test_o, locations, hospitalBeds, countryIncome, popDensity, False, e, i, s)

### Parts 5,6, and 7 for Random Forest

In [ ]:
param_dist_rf2 = {
    'n_estimators': Integer(10, 3000),
    'max_depth': Integer(3, 30),
    'min_samples_split': Integer(5, 30),
    'min_samples_leaf': Integer(5, 30),
    'max_features': Categorical(['auto', 'sqrt', 'log2']),
    'class_weight': Categorical(['balanced', 'balanced_subsample', None]),
    'ccp_alpha': Real(0.0, 0.1)}

rf = RandomForestClassifier(random_state=42)

In [ ]:
X_cols = train.dropna()
y_cols = X_cols['outcome_group']
X_cols = X_cols.drop(columns=['outcome_group'])

In [ ]:
# Perform the search 5 times using 50 iterations each time
# Save the best model each time and the evaluation metrics such as macro f1 score, accuracy, and roc auc score, and f1 score for the deceased class

best_models = []
best_scores = []

scoring = {'accuracy': make_scorer(accuracy_score),
           'f1': make_scorer(f1_score, average='macro'),
           'f1_deceased': make_scorer(f1_score, labels=[0], average='macro'),
           'f1_hospitalized': make_scorer(f1_score, labels=[1], average='macro'),
           'f1_nonhospitalized': make_scorer(f1_score, labels=[2], average='macro')     
           }

for i in range(5):
    search = BayesSearchCV(rf, param_dist_rf2, n_iter=15, random_state=dt.datetime.now().microsecond, cv=5, scoring=scoring, n_jobs=-1, verbose=2, refit='f1', return_train_score=True)
    search.fit(X_cols, y_cols)
    best_models.append(search.best_estimator_)
    print(f'Best params: {search.best_params_}')

    # Use the best model to perform 5 fold cross validation
    cv = KFold(n_splits=5, shuffle=True)
    scores = cross_validate(search.best_estimator_, X_cols, y_cols, cv=cv, scoring=scoring, n_jobs=-1)
    print(f'Cross validation scores: {scores}')
    best_scores.append(scores)

In [ ]:
# Save the best models with the mean cross validation scores for macro f1, accuracy, and f1 score for the deceased class
# Putting them in a dataframe and then saving as a text file

best_models_df = pd.DataFrame(pd.Series(best_models))
# Rename the column as Model
best_models_df.columns = ['Model']
best_scores_df = pd.DataFrame(best_scores)
best_models_df['Mean Macro F1'] = best_scores_df['test_f1'].mean()
best_models_df['Mean Accuracy'] = best_scores_df['test_accuracy'].mean()
best_models_df['Mean F1 Deceased'] = best_scores_df['test_f1_deceased'].mean()

# Sort by mean macro f1 score
best_models_df = best_models_df.sort_values('Mean Macro F1', ascending=False)

# Save df as text file
best_models_df.to_csv('randomforest_tuning.txt', sep='\t')

In [ ]:
# Select the best model based on the best f1 score
best_model = best_models_df.iloc[0]['Model']

# Store both training and testing metrics
train_metrics = []
test_metrics = []

# Perform 5 fold cross validation on the best model

kf = KFold(5, shuffle=True, random_state=42)
kf.get_n_splits(X_cols, y_cols)

for i, (train_index, test_index) in enumerate(kf.split(X_cols, y_cols)):

    X_train, X_test = X_cols.iloc[train_index], X_cols.iloc[test_index]
    y_train, y_test = y_cols.iloc[train_index], y_cols.iloc[test_index]

    best_model.fit(X_train, y_train)

    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)

    # predict probabilities
    y_pred_train_prob = best_model.predict_proba(X_train)
    y_pred_test_prob = best_model.predict_proba(X_test)

    train_metrics.append({'accuracy': accuracy_score(y_train, y_pred_train),
                          'f1_macro': f1_score(y_train, y_pred_train, average='macro'),
                          'f1_deceased': f1_score(y_train, y_pred_train, labels=[0], average='macro'),
                          'f1_hospitalized': f1_score(y_train, y_pred_train, labels=[1], average='macro'),
                          'f1_nonhospitalized': f1_score(y_train, y_pred_train, labels=[2], average='macro'),
                          'roc_auc': roc_auc_score(y_train, y_pred_train_prob, multi_class='ovr')})

    test_metrics.append({'accuracy': accuracy_score(y_test, y_pred_test),
                          'f1_macro': f1_score(y_test, y_pred_test, average='macro'),
                          'f1_deceased': f1_score(y_test, y_pred_test, labels=[0], average='macro'),
                          'f1_hospitalized': f1_score(y_test, y_pred_test, labels=[1], average='macro'),
                          'f1_nonhospitalized': f1_score(y_test, y_pred_test, labels=[2], average='macro'),
                          'roc_auc': roc_auc_score(y_test, y_pred_test_prob, multi_class='ovr')})
    
train_metrics = pd.DataFrame(train_metrics)
test_metrics = pd.DataFrame(test_metrics)

train_metrics.mean(), test_metrics.mean()


### Predicting test set

In [ ]:
best_model = RandomForestClassifier(class_weight='balanced_subsample', max_depth=30,
                       max_features='auto', min_samples_leaf=5,
                       min_samples_split=5, n_estimators=3000, random_state=42)

In [ ]:
# Fit the best model on the entire training set
best_model.fit(X_cols, y_cols)

# Predict the test set
y_pred = best_model.predict(test)

In [ ]:
plt.style.use('ggplot')
# Plotting the distribution of the train and test outcomes
# Normalize the train and test outcome groups
y_norm = y_cols.value_counts(normalize=True).sort_index().values
y_pred_norm = pd.Series(y_pred).value_counts(normalize=True).sort_index().values

# Plotting the normalized train and test outcome groups on the same plot
plt.figure(figsize=(5, 3))
plt.bar([0,1,2],y_norm, label='Train', alpha=0.5)
plt.bar([0,1,2],y_pred_norm, label='Test', alpha=0.5)
plt.legend()
plt.title('Train and Test Outcome Group')
plt.tight_layout()
plt.savefig('plots/train_test_outcome_group.svg')
plt.show()

# Computing KL Divergence between the train and test outcome groups
kl_div = entropy(pk=y_pred_norm, qk=y_norm)
print(f'KL Divergence: {kl_div:.3f}')


### Creating submission file



In [ ]:
# Create a submission file
create_submission_file(y_pred, 'submission_rf.csv')

# MLP

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_cols, y_cols)
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_smote, y_smote = sm.fit_resample(X_cols, y_cols)

from sklearn.metrics import make_scorer
def f1_deceased(y_true, y_pred):
    return f1_score(y_true, y_pred, average=None)[0]

custom_scorer = make_scorer(f1_deceased)

In [ ]:
#tune hyperparameters

from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,75), (100,50), (100,25),
                       (150,100), (150,50), (100,75,50), 
                       (100,50,25),(150,75,50),(200,100,50),
                      (200,150,100),(150,100,25),(150,125,100)],
'alpha': [0.1,0.01,0.5,1,.001],
'activation': ["logistic", "relu", "tanh", "identity"],
'solver': ["lbfgs", "sgd", "adam"],
'learning_rate_init':[0.001,0.0001]
}

mlp = RandomizedSearchCV(MLPClassifier(), parameters,verbose=3,n_jobs=-1,n_iter=25,scoring={'f1_deceased': custom_scorer},refit='f1_deceased')

In [ ]:
mlp.fit(X_cols,y_cols)
mlp_best = mlp.best_estimator_ #show best hyperparameters

In [ ]:
print("f1_deceased: ",f1_score(y_test, mlp_best.predict(X_test),average=None)[0],
      "\nf1_hospitalized: ",f1_score(y_test, mlp_best.predict(X_test),average=None)[1],
      "\nf1_nonhospitalized: ",f1_score(y_test, mlp_best.predict(X_test),average=None)[2],
"\nf1_macro: ",f1_score(y_test, mlp_best.predict(X_test), average='macro'))

# SVM

In [ ]:
X_cols = train[train.columns[0:-1]]
y_cols = train[train.columns[-1]]
X_train, X_valid, y_train, y_valid = train_test_split(X_cols, y_cols, test_size=0.2, random_state=42, stratify=y_cols)

In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)

X_smote = X_train
y_smote = y_train

print("Class distribution before SMOTE:")
print(y_smote.value_counts())
# Apply SMOTE to training data
X_resampled, y_resampled = smote.fit_resample(X_smote, y_smote)

# Check class distribution after SMOTE
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 'scale', 'auto'],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

svm_classifier = SVC(class_weight='balanced')

print("Starting grid search")
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='accuracy', verbose=1)
print("Grid searching")
grid_search.fit(X_train, y_train)
print("Finished grid search")
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

# Evaluate the model with the best hyperparameters
best_model_svm = grid_search.best_estimator_
accuracy = best_model_svm.score(X_valid, y_valid)
print("Accuracy on validation set:", accuracy)

In [ ]:
# Define the number of folds for cross-validation
k = 5

# Initialize variables to store evaluation metrics
avg_accuracy = 0
avg_f1 = 0
avg_f1_deceased = 0
avg_f1_hospitalized = 0
avg_f1_nonhospitalized = 0

# Initialize the cross-validator
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Iterate over each fold
for i, (train_index, test_index) in enumerate(kf.split(X_resampled, y_resampled)):
    # Split the data into training and testing sets for this fold
    X_train_cv, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train_cv, y_test = y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    
    # Fit the SVM model on the training data
    best_model_svm.fit(X_train_cv, y_train_cv)
    
    # Predict labels for the testing data
    y_pred = best_model_svm.predict(X_test)
    
    # Calculate evaluation metrics for this fold
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    f1_scores = f1_score(y_test, y_pred, average=None)
    
    # Accumulate evaluation metrics for averaging
    avg_accuracy += accuracy
    avg_f1 += f1
    avg_f1_deceased += f1_scores[0]
    avg_f1_hospitalized += f1_scores[1]
    avg_f1_nonhospitalized += f1_scores[2]
    
    print(f'Fold {i+1} - Accuracy: {accuracy:.4f} - F1: {f1:.4f} - F1 Deceased: {f1_score(y_test, y_pred, average=None)[0]}')

# Calculate average evaluation metrics across all folds
avg_accuracy /= k
avg_f1 /= k
avg_f1_deceased /= k
avg_f1_hospitalized /= k
avg_f1_nonhospitalized /= k

print(f'Average across {k}-fold CV - Accuracy: {avg_accuracy:.4f} - F1: {avg_f1:.4f} - F1 Deceased: {avg_f1_deceased:.4f} - F1 Hospitalized: {avg_f1_hospitalized:.4f} - F1 Nonhospitalized: {avg_f1_nonhospitalized:.4f}')

In [ ]:
y_train_pred = best_model_svm.predict(X_train)
y_val_pred = best_model_svm.predict(X_valid)

# Calculate evaluation metrics for training set
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

# Calculate evaluation metrics for validation set
val_accuracy = accuracy_score(y_valid, y_val_pred)
val_precision = precision_score(y_valid, y_val_pred, average='weighted')
val_recall = recall_score(y_valid, y_val_pred, average='weighted')
val_f1 = f1_score(y_valid, y_val_pred, average='weighted')
val_f1_per_class = f1_score(y_valid, y_val_pred, average=None)

# Print the performance metrics
print("Training Set Performance:")
print(f"Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-score: {train_f1:.4f}")

print("\nValidation Set Performance:")
print(f"Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-score: {val_f1:.4f}")
print("F1 Score per class: (in order deceased, hospitalized, nonhospitalized)", val_f1_per_class)

# Evaluate the model on the train set
print('\nTraining Report:\n', classification_report(y_train, y_train_pred))
 
# Evaluate the model on the test set
print('\nValidation report:\n', classification_report(y_valid, y_val_pred))

In [ ]:
# Compute the confusion matrix
cm_train = confusion_matrix(y_train, y_train_pred)
cm_val = confusion_matrix(y_valid, y_val_pred)

# Plot the confusion matrix
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.heatmap(cm_train, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Training Set)')

plt.subplot(1, 2, 2)
sns.heatmap(cm_val, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Validation Set)')

plt.tight_layout()
plt.show()